In [65]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold # import KFold
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import validation_curve
seed=111

### Loading data

In [48]:
r1=pd.read_pickle('/Users/Christian/Dropbox/Universitet/TSODAS/TSDS-gruppe-2019/Exam/prices_0_200.pkl')
r2=pd.read_pickle('/Users/Christian/Dropbox/Universitet/TSODAS/TSDS-gruppe-2019/Exam/prices_200_578.pkl')
r3=pd.read_pickle('/Users/Christian/Dropbox/Universitet/TSODAS/TSDS-gruppe-2019/Exam/prices_577_777.pkl')
r4=pd.read_pickle('/Users/Christian/Dropbox/Universitet/TSODAS/TSDS-gruppe-2019/Exam/prices_777_1000.pkl')
r5=pd.read_pickle('/Users/Christian/Dropbox/Universitet/TSODAS/TSDS-gruppe-2019/Exam/prices_1000_1500.pkl')
r6=pd.read_pickle('/Users/Christian/Dropbox/Universitet/TSODAS/TSDS-gruppe-2019/Exam/prices_1500_2000.pkl')

In [61]:
print(len(r1), len(r2), len(r3), len(r4), len(r5), len(r6))
r=r1.append(r2)
r=r.append(r3)
r=r.append(r4)
r=r.append(r5)
r=r.append(r6)
r=r.drop(r['price']=='-')
r=r.drop_duplicates(['origin','destination'])
r['price']=r['price'].str[4:].str.replace('.','')
r['label']= r['label'].astype('category')
r['alternative']= r['alternative'].astype('category')
r["alternative_cat"] = r['alternative'].cat.codes
r["label_cat"] = r['label'].cat.codes
r['price']=pd.to_numeric(r.price, errors='coerce')


200 400 200 222 500 500


In [62]:
r

,alternative,day,destination,label,origin,price,alternative_cat,label_cat
2,NaN,0,PHX,Direkte,SMF,1003.0,-1,2
3,NaN,0,SAN,Direkte,SMF,570.0,-1,2
4,NaN,0,SEA,Direkte,SMF,905.0,-1,2
5,,0,SNA,1 omstigning,SMF,1094.0,0,1
6,NaN,0,LAS,Direkte,SNA,820.0,-1,2
7,Ingen,0,MDW,1 omstigning,SNA,2108.0,1,1
8,NaN,0,OAK,Direkte,SNA,1688.0,-1,2
9,NaN,0,PHX,Direkte,SNA,1096.0,-1,2
10,NaN,0,SJC,Direkte,SNA,1009.0,-1,2
11,,0,SMF,1 omstigning,SNA,1408.0,0,1


### Exploring data

### Estimating model

In [ ]:
pipe_linear = make_pipeline(Lasso(k=10,random_state=seed))
scores = cross_val_score(estimator=pipe_linear, X=, y= , cv=10)
param_range_1 = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
# Validation curve
train_scores, test_scores = validation_curve(
    estimator=pipe_lr,
    X=X_train,
    y=y_train,
    param_name='logisticregression__C',
    param_range=param_range,
    cv=10)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)
plt.plot(param_range, train_mean, color='blue', marker='o',
    markersize=5, label='training accuracy')
plt.fill_between(param_range, train_mean + train_std,
    train_mean - train_std, alpha=0.15,
    color='blue')
plt.plot(param_range, test_mean,
    color='green', linestyle='--',
    marker='s', markersize=5,
    label='validation accuracy')
plt.fill_between(param_range,
    test_mean + test_std,
    test_mean - test_std,
    alpha=0.15, color='green')
plt.grid()
plt.xscale('log')
plt.legend(loc='lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.8, 1.03])
plt.show()

In [ ]:
### Grid search
param_range_1 = [0.0001, 0.001, 0.01, 0.1,
    1.0, 10.0, 100.0, 1000.0]
param_grid = [{'svc__C': param_range,
    'svc__gamma': param_range,
    'svc__kernel': ['rbf']}]

### Estimating model

In [ ]:

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]]) # create an array
y = np.array([1, 2, 3, 4]) # Create another array
kf = KFold(n_splits=2) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf) 
KFold(n_splits=2, random_state=None, shuffle=False)

for train_index, test_index in kf.split(X):
    print(“TRAIN:”, train_index, “TEST:”, test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

scores = cross_val_score(model, df, y, cv=6)
print(“Cross-validated scores:”, scores)

# 
lm = linear_model.LinearRegression()


# Make cross validated predictions
predictions = cross_val_predict(model, df, y, cv=6)
plt.scatter(y, predictions)

accuracy = metrics.r2_score(y, predictions)
print(“Cross-Predicted Accuracy:”, accuracy)
